<h1> Feature Engineering </h1>

In this notebook, you will learn how to incorporate feature engineering into your pipeline.
<ul>
<li> Working with feature columns </li>
<li> Adding feature crosses in TensorFlow </li>
<li> Reading data from BigQuery </li>
<li> Creating datasets using Dataflow </li>
<li> Using a wide-and-deep model </li>
</ul>

In [ ]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [ ]:
!pip install --user google-cloud-bigquery==1.25.0

|████████████████████████████████| 174kB 3.2MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


Kindly ignore the deprecation warnings and incompatibility errors related to google-cloud-storage.

In [ ]:
!pip install --user apache-beam[gcp]==2.16.0 
!pip install --user httplib2==0.12.0 

     |████████████████████████████████| 2.9MB 4.8MB/s eta 0:00:01
     |████████████████████████████████| 51kB 7.2MB/s  eta 0:00:01
     |████████████████████████████████| 276kB 27.6MB/s eta 0:00:01
     |████████████████████████████████| 225kB 49.4MB/s eta 0:00:01
     |████████████████████████████████| 81kB 10.5MB/s eta 0:00:01
     |████████████████████████████████| 460kB 34.3MB/s eta 0:00:01
     |████████████████████████████████| 92kB 10.2MB/s eta 0:00:01
     |████████████████████████████████| 153kB 29.2MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 18.2MB/s eta 0:00:01
     |████████████████████████████████| 58.1MB 127kB/s  eta 0:00:01     |█████████▋                      | 17.5MB 25.5MB/s eta 0:00:02
     |████████████████████████████████| 61kB 8.9MB/s  eta 0:00:01
     |████████████████████████████████| 143kB 34.3MB/s eta 0:00:01
     |████████████████████████████████| 174kB 34.5MB/s eta 0:00:01
     |████████████████████████████████| 122kB 36.9MB/s eta 0:00:01

NOTE: In the output of the above cell you may ignore any WARNINGS or ERRORS related to the following:  "apache-beam", "pyarrow", "tensorflow-transform", "tensorflow-model-analysis", "tensorflow-data-validation", "joblib", "google-cloud-storage" etc.

If you get any related errors mentioned above please rerun the above cell.

**Note**: Restart your kernel to use updated packages.

In [ ]:
import tensorflow as tf
import apache_beam as beam
import shutil
print(tf.__version__)

2.3.0


<h2> 1. Environment variables for project and bucket </h2>

1. Your project id is the *unique* string that identifies your project (not the project name). You can find this from the GCP Console dashboard's Home page.  My dashboard reads:  <b>Project ID:</b> cloud-training-demos 
2.  Cloud training often involves saving and restoring model files. Therefore, we should <b>create a single-region bucket</b>. If you don't have a bucket already, I suggest that you create one from the GCP console (because it will dynamically check whether the bucket name you want is available) 
<b>Change the cell below</b> to reflect your Project ID and bucket name.


In [1]:
import os
PROJECT = 'cloud-training-demos'    # CHANGE THIS
BUCKET = 'cloud-training-demos' # REPLACE WITH YOUR BUCKET NAME. Use a regional bucket in the region you selected.
REGION = 'us-central1' # Choose an available region for Cloud AI Platform

In [ ]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.1' 

## ensure we're using python3 env
os.environ['CLOUDSDK_PYTHON'] = 'python3'

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## ensure we predict locally with our current Python environment
gcloud config set ml_engine/local_python `which python`

<h2> 2. Specifying query to pull the data </h2>

Let's pull out a few extra columns from the timestamp.

In [ ]:
def create_query(phase, EVERY_N):
  if EVERY_N == None:
    EVERY_N = 4 #use full dataset
    
  #select and pre-process fields
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  DAYOFWEEK(pickup_datetime) AS dayofweek,
  HOUR(pickup_datetime) AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  CONCAT(STRING(pickup_datetime), STRING(pickup_longitude), STRING(pickup_latitude), STRING(dropoff_latitude), STRING(dropoff_longitude)) AS key
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """
  
  #add subsampling criteria by modding with hashkey
  if phase == 'train': 
    query = "{} AND ABS(HASH(pickup_datetime)) % {} < 2".format(base_query,EVERY_N)
  elif phase == 'valid': 
    query = "{} AND ABS(HASH(pickup_datetime)) % {} == 2".format(base_query,EVERY_N)
  elif phase == 'test':
    query = "{} AND ABS(HASH(pickup_datetime)) % {} == 3".format(base_query,EVERY_N)
  return query
    
print(create_query('valid', 100)) #example query using 1% of data

Try the query above in https://bigquery.cloud.google.com/table/nyc-tlc:yellow.trips if you want to see what it does (ADD LIMIT 10 to the query!)

<h2> 3. Preprocessing Dataflow job from BigQuery </h2>

This code reads from BigQuery and saves the data as-is on Google Cloud Storage.  We can do additional preprocessing and cleanup inside Dataflow, but then we'll have to remember to repeat that prepreprocessing during inference. It is better to use tf.transform which will do this book-keeping for you, or to do preprocessing within your TensorFlow model. We will look at this in future notebooks. For now, we are simply moving data from BigQuery to CSV using Dataflow.

While we could read from BQ directly from TensorFlow (See: https://www.tensorflow.org/api_docs/python/tf/contrib/cloud/BigQueryReader), it is quite convenient to export to CSV and do the training off CSV.  Let's use Dataflow to do this at scale.

Because we are running this on the Cloud, you should go to the GCP Console (https://console.cloud.google.com/dataflow) to look at the status of the job. It will take several minutes for the preprocessing job to launch.

In [ ]:
%%bash
if gsutil ls | grep -q gs://${BUCKET}/taxifare/ch4/taxi_preproc/; then
  gsutil -m rm -rf gs://$BUCKET/taxifare/ch4/taxi_preproc/
fi

First, let's define a function for preprocessing the data

In [ ]:
import datetime

####
# Arguments:
#   -rowdict: Dictionary. The beam bigquery reader returns a PCollection in
#     which each row is represented as a python dictionary
# Returns:
#   -rowstring: a comma separated string representation of the record with dayofweek
#     converted from int to string (e.g. 3 --> Tue)
####
def to_csv(rowdict):
  days = ['null', 'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
  CSV_COLUMNS = 'fare_amount,dayofweek,hourofday,pickuplon,pickuplat,dropofflon,dropofflat,passengers,key'.split(',')
  rowdict['dayofweek'] = days[rowdict['dayofweek']]
  rowstring = ','.join([str(rowdict[k]) for k in CSV_COLUMNS])
  return rowstring


####
# Arguments:
#   -EVERY_N: Integer. Sample one out of every N rows from the full dataset.
#     Larger values will yield smaller sample
#   -RUNNER: 'DirectRunner' or 'DataflowRunner'. Specfy to run the pipeline
#     locally or on Google Cloud respectively. 
# Side-effects:
#   -Creates and executes dataflow pipeline. 
#     See https://beam.apache.org/documentation/programming-guide/#creating-a-pipeline
####
def preprocess(EVERY_N, RUNNER):
  job_name = 'preprocess-taxifeatures' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')
  print('Launching Dataflow job {} ... hang on'.format(job_name))
  OUTPUT_DIR = 'gs://{0}/taxifare/ch4/taxi_preproc/'.format(BUCKET)

  #dictionary of pipeline options
  options = {
    'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
    'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
    'job_name': 'preprocess-taxifeatures' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S'),
    'project': PROJECT,
    'runner': RUNNER,
    'num_workers' : 4,
    'max_num_workers' : 5
  }
  #instantiate PipelineOptions object using options dictionary
  opts = beam.pipeline.PipelineOptions(flags=[], **options)
  #instantantiate Pipeline object using PipelineOptions
  with beam.Pipeline(options=opts) as p:
      for phase in ['train', 'valid']:
        query = create_query(phase, EVERY_N) 
        outfile = os.path.join(OUTPUT_DIR, '{}.csv'.format(phase))
        (
          p | 'read_{}'.format(phase) >> beam.io.Read(beam.io.BigQuerySource(query=query))
            | 'tocsv_{}'.format(phase) >> beam.Map(to_csv)
            | 'write_{}'.format(phase) >> beam.io.Write(beam.io.WriteToText(outfile))
        )
  print("Done")

Now, let's run pipeline locally. This takes upto <b>5 minutes</b>.  You will see a message "Done" when it is done.

In [ ]:
preprocess(50*10000, 'DirectRunner') 

Launching Dataflow job preprocess-taxifeatures-201007-104302 ... hang on
Done


In [ ]:
%%bash
gsutil ls gs://$BUCKET/taxifare/ch4/taxi_preproc/

gs://qwiklabs-gcp-03-6f3d948638d3/taxifare/ch4/taxi_preproc/train.csv-00000-of-00005
gs://qwiklabs-gcp-03-6f3d948638d3/taxifare/ch4/taxi_preproc/train.csv-00001-of-00005
gs://qwiklabs-gcp-03-6f3d948638d3/taxifare/ch4/taxi_preproc/train.csv-00002-of-00005
gs://qwiklabs-gcp-03-6f3d948638d3/taxifare/ch4/taxi_preproc/train.csv-00003-of-00005
gs://qwiklabs-gcp-03-6f3d948638d3/taxifare/ch4/taxi_preproc/train.csv-00004-of-00005
gs://qwiklabs-gcp-03-6f3d948638d3/taxifare/ch4/taxi_preproc/valid.csv-00000-of-00002
gs://qwiklabs-gcp-03-6f3d948638d3/taxifare/ch4/taxi_preproc/valid.csv-00001-of-00002


## 4. Run Beam pipeline on Cloud Dataflow

Run pipeline on cloud on a larger sample size.

In [ ]:
%%bash
if gsutil ls | grep -q gs://${BUCKET}/taxifare/ch4/taxi_preproc/; then
  gsutil -m rm -rf gs://$BUCKET/taxifare/ch4/taxi_preproc/
fi

The following step will take <b>15-20 minutes.</b> Monitor job progress on the [Cloud Console in the Dataflow](https://console.cloud.google.com/dataflow) section

In [ ]:
preprocess(50*100, 'DataflowRunner') 


Launching Dataflow job preprocess-taxifeatures-201007-104302 ... hang on
Done


Once the job completes, observe the files created in Google Cloud Storage

In [ ]:
%%bash
gsutil ls -l gs://$BUCKET/taxifare/ch4/taxi_preproc/

49169783  2020-10-07T11:11:48Z  gs://qwiklabs-gcp-c8b7c0b514e76634/taxifare/ch4/taxi_preproc/train.csv-00000-of-00001
   116320  2020-10-07T11:06:21Z  gs://qwiklabs-gcp-c8b7c0b514e76634/taxifare/ch4/taxi_preproc/train.csv-00000-of-00005
   108804  2020-10-07T11:06:21Z  gs://qwiklabs-gcp-c8b7c0b514e76634/taxifare/ch4/taxi_preproc/train.csv-00001-of-00005
   116799  2020-10-07T11:06:21Z  gs://qwiklabs-gcp-c8b7c0b514e76634/taxifare/ch4/taxi_preproc/train.csv-00002-of-00005
   114697  2020-10-07T11:06:21Z  gs://qwiklabs-gcp-c8b7c0b514e76634/taxifare/ch4/taxi_preproc/train.csv-00003-of-00005
   116173  2020-10-07T11:06:21Z  gs://qwiklabs-gcp-c8b7c0b514e76634/taxifare/ch4/taxi_preproc/train.csv-00004-of-00005
 24666705  2020-10-07T11:12:01Z  gs://qwiklabs-gcp-c8b7c0b514e76634/taxifare/ch4/taxi_preproc/valid.csv-00000-of-00001
   114332  2020-10-07T11:06:20Z  gs://qwiklabs-gcp-c8b7c0b514e76634/taxifare/ch4/taxi_preproc/valid.csv-00000-of-00002
   107689  2020-10-07T11:06:20Z  gs://qwiklabs-gc

In [ ]:
%%bash
#print first 10 lines of first shard of train.csv
gsutil cat "gs://$BUCKET/taxifare/ch4/taxi_preproc/train.csv-00000-of-*" | head

11.3,Fri,13,-73.975687,40.76003,-73.999972,40.762227,2.0,2009-09-11 13:56:00.000000-73.975740.7640.7622-74
6.1,Wed,12,-73.979987,40.757182,-73.999998,40.748792,2.0,2009-12-16 12:26:39.000000-73.9840.757240.7488-74
21.3,Tue,17,-74.00001,40.72167,-73.933223,40.679975,2.0,2012-05-15 17:38:00.000000-7440.721740.68-73.9332
6.5,Sat,15,-73.992025,40.725997,-74.000012,40.72185,2.0,2012-08-18 15:13:00.000000-73.99240.72640.7219-74
7.0,Sun,21,-73.991645,40.74996,-74.000044,40.730599,2.0,2014-04-06 21:49:00.000000-73.991640.7540.7306-74
14.0,Tue,22,-73.9999771118164,40.72161102294922,-73.95999145507812,40.76264572143555,2.0,2015-06-09 
22:19:48.000000-7440.721640.7626-73.96
10.9,Tue,14,-73.999958,40.730595,-73.980769,40.763996,2.0,2009-01-13 14:35:50.000000-7440.730640.764-73.9808
10.9,Sat,18,-73.999975,40.717953,-73.99127,40.750233,2.0,2009-04-18 18:31:00.000000-7440.71840.7502-73.9913
5.7,Thu,1,-73.99998,40.738628,-73.981518,40.741005,2.0,2009-09-10 01:02:14.000000-7440.738640.741-73.9815
6.1,S

## 5. Develop model with new inputs

Download the first shard of the preprocessed data to enable local development.

In [ ]:
%%bash
if [ -d sample ]; then
  rm -rf sample
fi
mkdir sample
gsutil cat "gs://$BUCKET/taxifare/ch4/taxi_preproc/train.csv-00000-of-*" > sample/train.csv
gsutil cat "gs://$BUCKET/taxifare/ch4/taxi_preproc/valid.csv-00000-of-*" > sample/valid.csv

We have two new inputs in the INPUT_COLUMNS, three engineered features, and the estimator involves bucketization and feature crosses.

In [ ]:
%%bash
grep -A 20 "INPUT_COLUMNS =" taxifare/trainer/model.py

In [ ]:
%%bash
grep -A 50 "build_estimator" taxifare/trainer/model.py

In [ ]:
%%bash
grep -A 15 "add_engineered(" taxifare/trainer/model.py

Try out the new model on the local sample (this takes <b>5 minutes</b>) to make sure it works fine.

In [ ]:
%%bash
rm -rf taxifare.tar.gz taxi_trained
export PYTHONPATH=${PYTHONPATH}:${PWD}/taxifare
python -m trainer.task \
  --train_data_paths=${PWD}/sample/train.csv \
  --eval_data_paths=${PWD}/sample/valid.csv  \
  --output_dir=${PWD}/taxi_trained \
  --train_steps=10 \
  --job-dir=/tmp

INFO:tensorflow:Using config:
..................................................................................................
..................................................................................................
..................................................................................................
INFO:tensorflow:Loss for final step: 184.59918.


In [ ]:
%%bash
ls taxi_trained/export/exporter/

1602060307


You can use ```saved_model_cli``` to look at the exported signature. Note that the model doesn't need any of the engineered features as inputs. It will compute latdiff, londiff, euclidean from the provided inputs, thanks to the ```add_engineered``` call in the serving_input_fn.

In [ ]:
%%bash
model_dir=$(ls ${PWD}/taxi_trained/export/exporter | tail -1)
saved_model_cli show --dir ${PWD}/taxi_trained/export/exporter/${model_dir} --all

In [ ]:
%%writefile /tmp/test.json
{"dayofweek": "Sun", "hourofday": 17, "pickuplon": -73.885262, "pickuplat": 40.773008, "dropofflon": -73.987232, "dropofflat": 40.732403, "passengers": 2}

Writing /tmp/test.json


In [ ]:
%%bash
model_dir=$(ls ${PWD}/taxi_trained/export/exporter)
gcloud ai-platform local predict \
  --model-dir=${PWD}/taxi_trained/export/exporter/${model_dir} \
  --json-instances=/tmp/test.json

PREDICTIONS
[2.600170373916626]


## 6. Train on cloud

This will take <b> 10-15 minutes </b> even though the prompt immediately returns after the job is submitted. Monitor job progress on the [Cloud Console, in the AI Platform](https://console.cloud.google.com/mlengine) section and wait for the training job to complete.


In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/taxifare/ch4/taxi_trained
JOBNAME=lab4a_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ai-platform jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=${PWD}/taxifare/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=BASIC \
  --runtime-version 2.1 \
  --python-version 3.5 \
  -- \
  --train_data_paths="gs://$BUCKET/taxifare/ch4/taxi_preproc/train*" \
  --eval_data_paths="gs://${BUCKET}/taxifare/ch4/taxi_preproc/valid*"  \
  --train_steps=5000 \
  --output_dir=$OUTDIR

gs://qwiklabs-gcp-c8b7c0b514e76634/taxifare/ch4/taxi_trained us-central1 lab4a_201007_111601
jobId: lab4a_201007_111601 
 state: QUEUED
CommandException: 1 files/objects could not be removed.            
Job [lab4a_201007_111601] submitted successfully.
Your job is still active. You may view the status of your job with the command     
  $ gcloud ai-platform jobs describe lab4a_201007_111601        
or continue streaming the logs with the command      
  $ gcloud ai-platform jobs stream-logs lab4a_201007_111601

The RMSE is now 8.33249, an improvement over the 9.3 that we were getting ... of course, we won't know until we train/validate on a larger dataset. Still, this is promising. But before we do that, let's do hyper-parameter tuning.

<b>Use the Cloud Console link to monitor the job and wait till the job is done.</b>

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License